### CAPM Beta and Volatility Estimation 
Author : Osho Sharma 


This file includes code to find yearly CAPM beta, along with systematic and idiosyncratic risk using monthly stock return data from CRSP (1996-2023)

In [1]:
#import libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [58]:
# import data 
data = pd.read_csv('./MSF_1996_2023.csv')

/var/folders/qq/4_7zbq5x7dx8hhvz65nzsm740000gn/T/ipykernel_5658/3716030037.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./MSF_1996_2023.csv')


#### Data Cleaning and Wrangling

In [88]:
data_filtered = data.filter(items=['date', 'PERMCO', 'vwretd', 'RET', 'SICCD',])
data_filtered.rename(columns = {'vwretd': 'market_return','RET': 'firm_return', 'TICKER' : 'ticker', 'SICCD' : 'industry_code', 'PERMCO': 'company_id'}, inplace = True)
# ind_data = data_filtered.groupby('industry_code')

# convert all industry code to numeric and handle errors.
data_filtered['industr_code_numeric'] = pd.to_numeric(data_filtered['industry_code'], errors='coerce')  
data_filtered = data_filtered.dropna(subset=['industr_code_numeric'])
data_filtered['industry_code'] = data_filtered['industry_code'].astype(int)
data_filtered.drop(columns=['industr_code_numeric'], inplace=True)

# Industry ranges SIC code 
industry_ranges = { 
    'agriculture': (1,999), 
    'mining': (1000,1499),
    'construction': (1500,1799),
    'manufacturing': (2000,3999),
    'transportation': (4000,4999),
    'wholesale_trade': (5000,5199),
    'retail_trade': (5200,5999),
    'finance': (6000,6799),
    'services': (7000,8999),
    'public_admin': (9000,9999)
}

# add new column for industry_name using industry_ranges
data_filtered['industry_name'] = data_filtered['industry_code'].apply(lambda x: next((k for k, v in industry_ranges.items() if v[0] <= x <= v[1]), None))
data_filtered['year'] = data_filtered['date'].str[:4]



# Filter ind_data to include monthly rows for 10 unique companies per industry_name for each year
sampled_data = data_filtered.groupby(['industry_name', 'year']).apply(
    lambda group: group[group['company_id'].isin(group.drop_duplicates('company_id').head(10)['company_id'])]
).reset_index(drop=True)

#filtered_ind_data = ind_data.apply(lambda x: x.groupby(x['date'].str[:4]).apply(lambda y: y.head(10))).reset_index(drop=True)

#code to find total number of industries have less than 10 tickers
# ticker_count = ind_data['company_id'].nunique()
# less_than_10 = ticker_count[ticker_count < 10]
# print(ind_data['company_id'].unique().count(), less_than_10.count())

# dictionary of 10 random tickers for each industry
# sampled_tickers = ind_data.apply(lambda x : x['company_id'].drop_duplicates().sample(n= min(10, len(x['company_id']))).to_list()).to_dict()

# for data in sampled_tickers: 
#     print(data, sampled_tickers[data])
#     print() 

# sampled_data = data_filtered[data_filtered['company_id'].isin([item for sublist in sampled_tickers.values() for item in sublist])]

/var/folders/qq/4_7zbq5x7dx8hhvz65nzsm740000gn/T/ipykernel_5658/1684594872.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_ind_data = data_filtered.groupby(['industry_name', 'year']).apply(


(31245, 7)

#### Linear Regression for Beta Estimation